In [6]:
import boto3
from dotenv import load_dotenv
import os
import csv

load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_default_region = os.getenv("AWS_DEFAULT_REGION")

boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_default_region
)

dynamodb = boto3.client('dynamodb')

table_name = 'User-7kkcm5dn2rb77hst5nh7gbdisa-staging'

# Especificamos las columnas que queremos exportar
columns_to_extract = ['id', 'userId', 'createdAt']

with open('dynamodb_users.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    last_evaluated_key = None

    # Escribimos la cabecera del archivo CSV con las columnas seleccionadas
    writer.writerow(columns_to_extract)

    while True:
        if last_evaluated_key:
            response = dynamodb.scan(
                TableName=table_name,
                ExclusiveStartKey=last_evaluated_key
            )
        else:
            response = dynamodb.scan(TableName=table_name)

        for item in response['Items']:
            row = []
            # Iteramos solo sobre las columnas seleccionadas
            for column in columns_to_extract:
                value = item.get(column, {}).get('S', '')  # Si el valor es de tipo String (S)
                if not value:
                    value = item.get(column, {}).get('N', '')  # Si el valor es de tipo Number (N)
                if not value:
                    value = item.get(column, {}).get('B', '')  # Si el valor es de tipo Binary (B)
                row.append(value)

            # Escribimos la fila solo con las columnas seleccionadas
            writer.writerow(row)

        if 'LastEvaluatedKey' in response:
            last_evaluated_key = response['LastEvaluatedKey']
        else:
            break

print("Datos exportados a dynamodb_users.csv")



Datos exportados a dynamodb_users.csv
